In [ ]:
# ============================================================
# KMU Comovement — Pair Classifier Only
#  - (A,B) 9900쌍에 대해 공행성 확률 comov_prob 예측
#  - 아직 value 예측(회귀)은 안 함
# ============================================================

import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

import warnings
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)

DATA_DIR = Path(".")
TRAIN_PATH = DATA_DIR / "train.csv"
SUB_PATH   = DATA_DIR / "sample_submission.csv"

# 이전 제출물들 (pseudo-label용)
PSEUDO_FILES = [
    "v3_lightgbm_submission.csv",
    "submission 0.3.csv",
]


# ============================================================
# 1. 데이터 로드 + 월별 집계 (panel)
# ============================================================
print("▶ Loading data...")
train = pd.read_csv(TRAIN_PATH)
sub   = pd.read_csv(SUB_PATH)

assert {"item_id", "year", "month", "value"}.issubset(train.columns)
assert {"leading_item_id", "following_item_id"}.issubset(sub.columns)

# year-month → date
train["date"] = pd.to_datetime(
    dict(year=train["year"], month=train["month"], day=1)
)

# 월별 value 합
monthly = (
    train.groupby(["item_id", "date"], as_index=False)["value"]
         .sum()
)

# 패널: index=date, columns=item_id
panel = monthly.pivot(index="date", columns="item_id", values="value").sort_index()
print(f"  - panel shape = {panel.shape}")  # (n_months, n_items)

dates = panel.index
items = panel.columns.tolist()

# item별 hs4, hs2
if "hs4" in train.columns:
    item_hs4 = train.groupby("item_id")["hs4"].first().to_dict()
else:
    item_hs4 = {i: None for i in items}

def get_hs2(hs4):
    if pd.isna(hs4):
        return None
    s = str(hs4)
    return s[:2]

item_hs2 = {i: get_hs2(item_hs4.get(i, None)) for i in items}

# active ratio (얼마나 자주 등장하는지)
total_months = len(panel.index)
active_ratio = (panel.notna().sum(axis=0) / total_months).to_dict()

# 계절 패턴 (item별 month→평균 value → cosine similarity)
monthly_tmp = monthly.copy()
monthly_tmp["month_num"] = monthly_tmp["date"].dt.month
seasonal = monthly_tmp.groupby(["item_id", "month_num"])["value"].mean().unstack("month_num").fillna(0.0)
seasonal_norm = seasonal.div(seasonal.sum(axis=1).replace(0,1), axis=0)

def seasonal_sim(A, B):
    if (A not in seasonal_norm.index) or (B not in seasonal_norm.index):
        return 0.0
    vA = seasonal_norm.loc[A].values
    vB = seasonal_norm.loc[B].values
    num = float((vA * vB).sum())
    den = float(np.linalg.norm(vA) * np.linalg.norm(vB))
    return float(num / den) if den > 0 else 0.0


# ============================================================
# 2. 이전 제출물들에서 pair 등장 횟수 (appear_cnt)
# ============================================================
print("▶ Loading previous submissions for appear_cnt...")
pair_appear = defaultdict(int)

for fname in PSEUDO_FILES:
    p = DATA_DIR / fname
    if not p.exists():
        print(f"  - {fname} not found, skip.")
        continue
    df_sub = pd.read_csv(p)
    if not {"leading_item_id", "following_item_id"}.issubset(df_sub.columns):
        print(f"  - {fname} has wrong schema, skip.")
        continue
    uniq_pairs = df_sub[["leading_item_id", "following_item_id"]].drop_duplicates()
    for _, row in uniq_pairs.iterrows():
        A = row["leading_item_id"]
        B = row["following_item_id"]
        pair_appear[(A, B)] += 1
        # 방향성 있는 문제라 (B,A)는 별도로 계산


# ============================================================
# 3. pair-level feature 생성
# ============================================================
print("▶ Building pair-level features...")

MAX_LAG = 12

def compute_pair_stats(A, B, max_lag=MAX_LAG):
    """(A,B) 쌍에 대해 lag 1~max_lag 기준 공행성 관련 feature 계산"""
    if (A not in panel.columns) or (B not in panel.columns):
        return None

    sA = panel[A].astype(float)
    sB = panel[B].astype(float)

    # pct_change
    sA_p = sA.pct_change().replace([np.inf, -np.inf], np.nan)
    sB_p = sB.pct_change().replace([np.inf, -np.inf], np.nan)

    best_lag = None
    best_diff_corr = -999.0
    best_level_corr = 0.0
    best_n_overlap = 0

    for lag in range(1, max_lag+1):
        # A_t → B_{t+lag}
        a_cut = sA_p.iloc[:-lag]
        b_shift = sB_p.shift(-lag).iloc[:-lag]
        valid = a_cut.notna() & b_shift.notna()
        n_ov = int(valid.sum())
        if n_ov < 5:  # 최소 5개 이상 같이 있어야 의미 있다고 보자
            continue

        x = a_cut[valid]
        y = b_shift[valid]
        if x.nunique() <= 1 or y.nunique() <= 1:
            continue

        corr = np.corrcoef(x, y)[0, 1]
        if np.isnan(corr):
            continue

        # best_lag는 diff_corr 기준 최대값
        if corr > best_diff_corr:
            best_diff_corr = float(corr)
            best_lag = lag
            # level corr은 value 기준으로 재계산
            a_val = sA.iloc[:-lag]
            b_val = sB.shift(-lag).iloc[:-lag]
            v2 = a_val.notna() & b_val.notna()
            if v2.sum() >= 5:
                xv = a_val[v2]
                yv = b_val[v2]
                if xv.nunique() > 1 and yv.nunique() > 1:
                    lc = np.corrcoef(xv, yv)[0, 1]
                    best_level_corr = float(lc) if not np.isnan(lc) else 0.0
                else:
                    best_level_corr = 0.0
            else:
                best_level_corr = 0.0
            best_n_overlap = n_ov

    # spike_jaccard
    # pct_change 기준으로 |변화율| 큰 달을 spike로.
    # threshold는 상위 20% 정도로 설정 (데이터에 따라 다를 수 있음)
    def spike_mask(s_p):
        s_abs = s_p.abs()
        if s_abs.notna().sum() == 0:
            return pd.Series(False, index=s_p.index)
        thr = s_abs.quantile(0.8)
        return s_abs >= thr

    maskA = spike_mask(sA_p)
    maskB = spike_mask(sB_p)
    inter = (maskA & maskB).sum()
    union = (maskA | maskB).sum()
    spike_jacc = float(inter / union) if union > 0 else 0.0

    # seasonal similarity
    seas_sim = seasonal_sim(A, B)

    # hs similarity
    hA = item_hs4.get(A, None)
    hB = item_hs4.get(B, None)
    hs2A = item_hs2.get(A, None)
    hs2B = item_hs2.get(B, None)

    same_hs4 = int(hA is not None and hB is not None and str(hA) == str(hB))
    same_hs2 = int(hs2A is not None and hs2B is not None and str(hs2A) == str(hs2B))

    # hs_similarity 숫자형 (대략적인 스코어)
    if same_hs4:
        hs_sim = 1.0
    elif same_hs2:
        hs_sim = 0.7
    else:
        hs_sim = 0.0

    # active ratio
    actA = float(active_ratio.get(A, 0.0))
    actB = float(active_ratio.get(B, 0.0))

    return dict(
        best_lag=best_lag if best_lag is not None else 0,
        diff_corr=best_diff_corr if best_lag is not None else 0.0,
        level_corr=best_level_corr if best_lag is not None else 0.0,
        n_overlap=best_n_overlap,
        spike_jaccard=spike_jacc,
        seasonal_sim=seas_sim,
        same_hs4=same_hs4,
        same_hs2=same_hs2,
        hs_similarity=hs_sim,
        active_A=actA,
        active_B=actB,
    )


pairs = sub[["leading_item_id", "following_item_id"]].drop_duplicates()
features = []

for idx, row in pairs.iterrows():
    A = row["leading_item_id"]
    B = row["following_item_id"]
    stats = compute_pair_stats(A, B, max_lag=MAX_LAG)
    if stats is None:
        # panel에 없는 item이 있을 경우
        stats = dict(
            best_lag=0,
            diff_corr=0.0,
            level_corr=0.0,
            n_overlap=0,
            spike_jaccard=0.0,
            seasonal_sim=0.0,
            same_hs4=0,
            same_hs2=0,
            hs_similarity=0.0,
            active_A=float(active_ratio.get(A, 0.0)),
            active_B=float(active_ratio.get(B, 0.0)),
        )
    stats["leading_item_id"] = A
    stats["following_item_id"] = B
    stats["appear_cnt"] = pair_appear.get((A, B), 0)
    features.append(stats)

pair_df = pd.DataFrame(features)
print(f"  - pair_df shape: {pair_df.shape}")


# ============================================================
# 4. pseudo-label 생성 (signal_score 기반)
# ============================================================
print("▶ Building pseudo-labels...")

# 결측 채우기
for col in ["diff_corr", "level_corr", "spike_jaccard", "seasonal_sim", "hs_similarity"]:
    pair_df[col] = pair_df[col].fillna(0.0)

# 공행성 신호 점수 (가중합은 필요하면 조정 가능)
pair_df["signal_score"] = (
    0.5 * pair_df["diff_corr"] +
    0.2 * pair_df["level_corr"] +
    0.2 * pair_df["spike_jaccard"] +
    0.1 * pair_df["seasonal_sim"] +
    0.1 * pair_df["hs_similarity"]
)

high_q = pair_df["signal_score"].quantile(0.80)  # 상위 20%
low_q  = pair_df["signal_score"].quantile(0.20)  # 하위 20%

pair_df["pseudo_label"] = -1  # -1은 학습에 사용하지 않음

# 강 양성: signal_score 상위 + 또는 이전 제출에서 등장
pos_mask = (
    (pair_df["signal_score"] >= high_q) |
    (pair_df["appear_cnt"] >= 1)
)

# 강 음성: signal_score 하위 + appear_cnt == 0
neg_mask = (
    (pair_df["signal_score"] <= low_q) &
    (pair_df["appear_cnt"] == 0)
)

pair_df.loc[pos_mask, "pseudo_label"] = 1
pair_df.loc[neg_mask, "pseudo_label"] = 0

n_pos = int((pair_df["pseudo_label"] == 1).sum())
n_neg = int((pair_df["pseudo_label"] == 0).sum())
n_used = int((pair_df["pseudo_label"] >= 0).sum())
print(f"  - #pseudo positive: {n_pos}")
print(f"  - #pseudo negative: {n_neg}")
print(f"  - pseudo-labeled pairs total: {n_used}")

pseudo_df = pair_df[pair_df["pseudo_label"] >= 0].reset_index(drop=True)


# ============================================================
# 5. LightGBM 분류기 학습 (공행성쌍 판별)
# ============================================================
print("▶ Training pair classification model (LightGBM)...")

CLF_PARAMS = dict(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=-1,
    subsample=0.9,
    colsample_bytree=0.9,
    min_data_in_leaf=20,
    reg_lambda=1.0,
    reg_alpha=0.0,
    objective="binary",
    random_state=SEED,
    force_row_wise=True,
    verbosity=-1,
)

clf_features = [
    "best_lag",
    "diff_corr",
    "level_corr",
    "n_overlap",
    "spike_jaccard",
    "seasonal_sim",
    "active_A",
    "active_B",
    "hs_similarity",
    "same_hs4",
    "same_hs2",
    # appear_cnt는 feature에서 제거! (라벨 만들 때만 사용)
]

X_clf = pseudo_df[clf_features].copy()
y_clf = pseudo_df["pseudo_label"].astype(int)

X_clf = X_clf.replace([np.inf, -np.inf], np.nan).fillna(0.0)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
oof_pred = np.zeros(len(X_clf))

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_clf, y_clf), 1):
    X_tr, X_va = X_clf.iloc[tr_idx], X_clf.iloc[va_idx]
    y_tr, y_va = y_clf.iloc[tr_idx], y_clf.iloc[va_idx]

    clf = LGBMClassifier(**CLF_PARAMS)
    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="binary_logloss",
        callbacks=[
            early_stopping(stopping_rounds=100, verbose=False),
            log_evaluation(period=0),
        ],
    )
    oof_pred[va_idx] = clf.predict_proba(X_va)[:, 1]

print(f"  - OOF AUC = {roc_auc_score(y_clf, oof_pred):.4f}")
print(f"  - OOF AP  = {average_precision_score(y_clf, oof_pred):.4f}")

# 전체 pseudo 라벨로 최종 학습
clf_final = LGBMClassifier(**CLF_PARAMS)
clf_final.fit(X_clf, y_clf)


# ============================================================
# 6. 전체 9900쌍에 대해 공행성 확률 예측
# ============================================================
print("▶ Predicting comovement probability for all pairs...")

X_all_pairs = pair_df[clf_features].copy()
X_all_pairs = X_all_pairs.replace([np.inf, -np.inf], np.nan).fillna(0.0)

pair_df["comov_prob"] = clf_final.predict_proba(X_all_pairs)[:, 1]
print(pair_df["comov_prob"].describe())

# 결과 저장: 공행성 확률 + 모든 pair-level feature까지 같이 저장
out_prob_path = "pair_comov_prob_full.csv"
pair_df.to_csv(out_prob_path, index=False)
print(f"▶ Saved pair features + prob: {out_prob_path}, shape={pair_df.shape}")



▶ Loading data...
  - panel shape = (43, 100)
▶ Loading previous submissions for appear_cnt...
▶ Building pair-level features...
  - pair_df shape: (9900, 14)
▶ Building pseudo-labels...
  - #pseudo positive: 4134
  - #pseudo negative: 1546
  - pseudo-labeled pairs total: 5680
▶ Training pair classification model (LightGBM)...
  - OOF AUC = 0.9585
  - OOF AP  = 0.9866
▶ Predicting comovement probability for all pairs...
count    9900.000000
mean        0.816455
std         0.360236
min         0.000094
25%         0.945612
50%         0.999971
75%         1.000000
max         1.000000
Name: comov_prob, dtype: float64
▶ Saved pair features + prob: pair_comov_prob_full.csv, shape=(9900, 17)


In [ ]:
# ============================================================
# KMU Comovement — Pair Classifier 결과 기반 회귀 + 제출 생성
#  - 입력: train.csv, sample_submission.csv, pair_comov_prob.csv
#  - 출력: submission.csv (선택된 공행성쌍에 대해 2025-08 value 예측)
# ============================================================

import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)

DATA_DIR = Path(".")
TRAIN_PATH = DATA_DIR / "train.csv"
SUB_PATH   = DATA_DIR / "sample_submission.csv"
PAIR_PROB_PATH = DATA_DIR / "pair_comov_prob_full.csv"

# 🔧 공행성쌍으로 쓸 상위 N개 (중요!)
TOP_N = 2400   # 1800~2600 사이 정도를 왔다갔다 하면서 튜닝해볼 수 있음

# LightGBM 회귀 파라미터
REG_PARAMS = dict(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=63,
    max_depth=-1,
    subsample=0.9,
    colsample_bytree=0.9,
    min_data_in_leaf=25,
    min_sum_hessian_in_leaf=1.0,
    reg_lambda=1.0,
    reg_alpha=0.1,
    objective="rmse",
    random_state=SEED,
    force_row_wise=True,
    verbosity=-1,
)

# ============================================================
# 1. 데이터 로드 + 월별 패널 생성
# ============================================================
print("▶ Loading data...")
train = pd.read_csv(TRAIN_PATH)
sub   = pd.read_csv(SUB_PATH)
pair_df = pd.read_csv(PAIR_PROB_PATH)

assert {"item_id", "year", "month", "value"}.issubset(train.columns)
assert {"leading_item_id","following_item_id","comov_prob"}.issubset(pair_df.columns)

# year-month → date
train["date"] = pd.to_datetime(
    dict(year=train["year"], month=train["month"], day=1)
)

# 월별 value 합
monthly = (
    train.groupby(["item_id","date"], as_index=False)["value"]
         .sum()
)

# 패널: index=date, columns=item_id
panel = monthly.pivot(index="date", columns="item_id", values="value").sort_index()
print(f"  - panel shape = {panel.shape}")  # (n_months, n_items)

dates = panel.index.to_list()
items = panel.columns.to_list()
n_months = len(dates)

# 최근 달이 2025-07인지 확인
print("  - date range:", dates[0], "~", dates[-1])

# ============================================================
# 2. comov_prob 기준 상위 N개 쌍 선택
# ============================================================
print("▶ Selecting top-N comovement pairs...")
pair_sorted = pair_df.sort_values("comov_prob", ascending=False).reset_index(drop=True)
candidates = pair_sorted.head(TOP_N).copy()

print(f"  - TOP_N = {TOP_N}")
print(f"  - selected candidate pairs: {len(candidates)}")

# best_lag가 없는 경우(0 또는 NaN)는 최소 1로 보정
candidates["best_lag"] = candidates["best_lag"].fillna(0).astype(int)
candidates.loc[candidates["best_lag"] <= 0, "best_lag"] = 1

# ============================================================
# 3. 회귀 학습용 데이터셋 생성
# ============================================================
print("▶ Building regression dataset for B(t+1) prediction...")

rows = []

for idx, row in candidates.iterrows():
    A = row["leading_item_id"]
    B = row["following_item_id"]
    lag = int(row["best_lag"])

    if (A not in panel.columns) or (B not in panel.columns):
        continue

    sA = panel[A].fillna(0.0).values.astype(float)
    sB = panel[B].fillna(0.0).values.astype(float)

    # 로그 스케일값 (스케일 안정화용)
    sA_log = np.log1p(np.clip(sA, 0, None))
    sB_log = np.log1p(np.clip(sB, 0, None))

    for t in range(n_months - 1):
        # target은 B_{t+1}
        if t + 1 >= n_months:
            continue
        # lag, 과거 3개월 등을 확보해야 하므로 인덱스 체크
        if t - lag < 0 or t - 1 < 0:
            continue

        # B 현재/과거
        b_t      = sB_log[t]
        b_t_1    = sB_log[t-1]
        b_t_2    = sB_log[t-2] if t >= 2 else 0.0
        b_t_3    = sB_log[t-3] if t >= 3 else 0.0

        # A 리드 값 (lag 반영: A_{t-lag})
        a_t_lag  = sA_log[t - lag]
        a_t_lag1 = sA_log[t - lag - 1] if (t - lag - 1) >= 0 else 0.0

        # 이동 평균 (log 값 기준)
        b_ma3 = sB_log[max(0, t-2):t+1].mean()
        b_ma6 = sB_log[max(0, t-5):t+1].mean()
        a_ma3 = sA_log[max(0, t-2):t+1].mean()

        # MoM (선형 스케일 기준에서 근사)
        b_prev_lin = sB[t-1] if t-1 >= 0 else 0.0
        b_curr_lin = sB[t]
        if b_prev_lin > 0:
            b_mom = (b_curr_lin - b_prev_lin) / (b_prev_lin + 1e-6)
        else:
            b_mom = 0.0

        a_prev_lin = sA[t-lag-1] if (t-lag-1) >= 0 else 0.0
        a_curr_lin = sA[t-lag] if (t-lag) >= 0 else 0.0
        if a_prev_lin > 0:
            a_mom = (a_curr_lin - a_prev_lin) / (a_prev_lin + 1e-6)
        else:
            a_mom = 0.0

        # 달/연 효과
        dt_t = dates[t]   # pandas.Timestamp
        month = dt_t.month
        year_effect = dt_t.year - 2022  # 2022 기준 offset

        # 타깃(B_{t+1}) log1p
        target_lin = sB[t+1]
        target_log = np.log1p(np.clip(target_lin, 0, None))

        rows.append({
            "leading_item_id": A,
            "following_item_id": B,
            "date_idx": t,   # 시간 순서를 위한 인덱스
            "b_t": b_t,
            "b_t_1": b_t_1,
            "b_t_2": b_t_2,
            "b_t_3": b_t_3,
            "a_t_lag": a_t_lag,
            "a_t_lag1": a_t_lag1,
            "b_ma3": b_ma3,
            "b_ma6": b_ma6,
            "a_ma3": a_ma3,
            "b_mom": b_mom,
            "a_mom": a_mom,
            "best_lag": float(lag),
            "comov_prob": float(row["comov_prob"]),
            "month": month,
            "year_effect": year_effect,
            "target": target_log,
        })

reg_df = pd.DataFrame(rows)
print(f"  - regression dataset shape: {reg_df.shape}")

if reg_df.empty:
    raise RuntimeError("회귀 학습 데이터가 비어 있습니다. TOP_N 또는 lag 조건을 확인하세요.")

# ============================================================
# 4. 회귀 모델 학습 (간단한 시간기반 CV)
# ============================================================
print("▶ Training regression model (LightGBM)...")

feature_cols = [
    "b_t","b_t_1","b_t_2","b_t_3",
    "a_t_lag","a_t_lag1",
    "b_ma3","b_ma6","a_ma3",
    "b_mom","a_mom",
    "best_lag",
    "comov_prob",
    "month","year_effect",
]

X_reg = reg_df[feature_cols].replace([np.inf,-np.inf], np.nan).fillna(0.0)
y_reg = reg_df["target"].values
date_idx = reg_df["date_idx"].values

# 날짜 인덱스를 기준으로 단순한 시간 기반 KFold 나누기
unique_t = sorted(np.unique(date_idx))
n_splits = 4
splits = np.array_split(unique_t, n_splits)

oof_pred = np.zeros(len(reg_df))

for fold, va_times in enumerate(splits, 1):
    va_mask = np.isin(date_idx, va_times)
    tr_mask = ~va_mask

    X_tr, X_va = X_reg[tr_mask], X_reg[va_mask]
    y_tr, y_va = y_reg[tr_mask], y_reg[va_mask]

    reg = LGBMRegressor(**REG_PARAMS)
    reg.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="rmse",
        callbacks=[
            early_stopping(stopping_rounds=100, verbose=False),
            log_evaluation(period=0),
        ],
    )
    oof_pred[va_mask] = reg.predict(X_va)

rmse_oof = np.sqrt(np.mean((oof_pred - y_reg)**2))
print(f"  - OOF RMSE (log1p scale) = {rmse_oof:.4f}")

# 전체 데이터로 최종 학습
reg_final = LGBMRegressor(**REG_PARAMS)
reg_final.fit(X_reg, y_reg, callbacks=[log_evaluation(period=0)])

# ============================================================
# 5. 2025-08 예측 (t=2025-07 기준)
# ============================================================
print("▶ Predicting 2025-08 values for selected pairs...")

t_last = n_months - 1  # 마지막 달 index (아마 2025-07)
dt_last = dates[t_last]
print("  - last observed month:", dt_last)

pred_dict = {}

for idx, row in candidates.iterrows():
    A = row["leading_item_id"]
    B = row["following_item_id"]
    lag = int(row["best_lag"])
    prob = float(row["comov_prob"])

    if (A not in panel.columns) or (B not in panel.columns):
        continue

    sA = panel[A].fillna(0.0).values.astype(float)
    sB = panel[B].fillna(0.0).values.astype(float)
    sA_log = np.log1p(np.clip(sA, 0, None))
    sB_log = np.log1p(np.clip(sB, 0, None))

    t = t_last  # t 기준으로 t+1(=2025-08)을 예측
    if t - lag < 0 or t - 1 < 0:
        # lag 때문에 feature가 안나오면 B_last 그대로 사용
        yhat_lin = float(sB[t])
        yhat = max(0.0, yhat_lin)
        yhat = int(round(yhat))
        pred_dict[(A,B)] = yhat
        continue

    b_t      = sB_log[t]
    b_t_1    = sB_log[t-1]
    b_t_2    = sB_log[t-2] if t >= 2 else 0.0
    b_t_3    = sB_log[t-3] if t >= 3 else 0.0

    a_t_lag  = sA_log[t - lag]
    a_t_lag1 = sA_log[t - lag - 1] if (t - lag - 1) >= 0 else 0.0

    b_ma3 = sB_log[max(0, t-2):t+1].mean()
    b_ma6 = sB_log[max(0, t-5):t+1].mean()
    a_ma3 = sA_log[max(0, t-2):t+1].mean()

    b_prev_lin = sB[t-1] if t-1 >= 0 else 0.0
    b_curr_lin = sB[t]
    if b_prev_lin > 0:
        b_mom = (b_curr_lin - b_prev_lin) / (b_prev_lin + 1e-6)
    else:
        b_mom = 0.0

    a_prev_lin = sA[t-lag-1] if (t-lag-1) >= 0 else 0.0
    a_curr_lin = sA[t-lag] if (t-lag) >= 0 else 0.0
    if a_prev_lin > 0:
        a_mom = (a_curr_lin - a_prev_lin) / (a_prev_lin + 1e-6)
    else:
        a_mom = 0.0

    month = 8                 # 예측 대상: 8월
    year_effect = 2025 - 2022

    feat_vec = {
        "b_t": b_t,
        "b_t_1": b_t_1,
        "b_t_2": b_t_2,
        "b_t_3": b_t_3,
        "a_t_lag": a_t_lag,
        "a_t_lag1": a_t_lag1,
        "b_ma3": b_ma3,
        "b_ma6": b_ma6,
        "a_ma3": a_ma3,
        "b_mom": b_mom,
        "a_mom": a_mom,
        "best_lag": float(lag),
        "comov_prob": prob,
        "month": month,
        "year_effect": year_effect,
    }

    X_test = pd.DataFrame([feat_vec])[feature_cols].replace([np.inf,-np.inf], np.nan).fillna(0.0)
    y_log = float(reg_final.predict(X_test)[0])
    y_lin = np.expm1(y_log)
    if not np.isfinite(y_lin):
        y_lin = float(sB[t])

    y_lin = max(0.0, y_lin)
    yhat = int(round(y_lin))

    pred_dict[(A,B)] = yhat

print(f"  - predicted pairs: {len(pred_dict)}")

# ============================================================
# 6. submission.csv 생성 (선택된 쌍만)
# ============================================================
print("▶ Building submission file...")

rows_out = []
cand_keys = set(pred_dict.keys())

for _, row in sub.iterrows():
    A = row["leading_item_id"]
    B = row["following_item_id"]
    key = (A,B)
    if key not in cand_keys:
        # 공행성쌍으로 선택되지 않은 건 제출하지 않음
        continue
    rows_out.append({
        "leading_item_id": A,
        "following_item_id": B,
        "value": pred_dict[key],
    })

sub_out = pd.DataFrame(rows_out,
                       columns=["leading_item_id","following_item_id","value"])
sub_out.to_csv("submission.csv", index=False)
print(f"▶ Saved submission: submission.csv shape= {sub_out.shape}")
print(sub_out.head())


▶ Loading data...
  - panel shape = (43, 100)
  - date range: 2022-01-01 00:00:00 ~ 2025-07-01 00:00:00
▶ Selecting top-N comovement pairs...
  - TOP_N = 2400
  - selected candidate pairs: 2400
▶ Building regression dataset for B(t+1) prediction...
  - regression dataset shape: (83338, 19)
▶ Training regression model (LightGBM)...
  - OOF RMSE (log1p scale) = 1.8948
▶ Predicting 2025-08 values for selected pairs...
  - last observed month: 2025-07-01 00:00:00
  - predicted pairs: 2400
▶ Building submission file...
▶ Saved submission: submission.csv shape= (2400, 3)
  leading_item_id following_item_id    value
0        DEWLVASR          AHMDUILJ    81309
1        DEWLVASR          XMKRPGLB   724545
2        DEWLVASR          ZKENOUDA   354698
3        DEWLVASR          LLHREMKS    40340
4        DEWLVASR          WPQXWHYO  9139203
